In [19]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import folium

import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px

%matplotlib inline


In [18]:
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"

df = pd.read_csv(URL)
print('Data downloaded and read into a dataframe!')

Data downloaded and read into a dataframe!


In [30]:
import dash
import dash_html_components as html
import dash_core_components as dcc

app = dash.Dash(__name__)

year_list = [i for i in range(1980, 2024)]

app.layout = html.Div(children=[
    html.H1("Automobile Sales Statistics Dashboard", 
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 26}),
    
        dcc.Dropdown(
            id='dropdown-statistics',
            options=[
                {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
            ],
            placeholder='Select a report type',
            value="Select Statistics",
            style={'width': '60%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
        ),
        dcc.Dropdown(
            id="select-year",
            options=[{'label': str(i), 'value': i} for i in year_list],
            placeholder="Select year",
            value="Select year",
            style={'width': '60%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
        ),
        html.Div([
            html.Div(id="output-container", className="chart-grid",style={
                 'display': 'flex', 
                 'border': '5px solid black', 
                 'width': '80%', 
                 'height': '200px', 
                 'margin': '20px auto',
                 'background-color': '#f0f0f0',
                 'justify-content': 'center',
                 'align-items': 'center'
             })
    ])
])

# layout Ends
@app.callback(
        Output(component_id="select-year",component_property="disabled"),
        Input(component_id="dropdown-statistics",component_property="value"))
def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics': 
        return False
    else: 
        return True




@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'), Input(component_id='select-year', component_property='value')])

def update_output_container(selected_statistics, selected_year):
    if selected_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = df[df['Recession'] == 1]
        return html.Div([
            html.H3("Recession Period Sales Data")])
    
    elif selected_statistics == "Yearly Statistics" :
        yearly_data= df[df['Recession'] == 0]
        return html.Div([
            html.H3("Recession Period Sales Data")])
        
        














if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import pandas as pd

# Example data
data = pd.DataFrame({
    'Year': range(1980, 2024),
    'Sales': range(1000, 1000 + 44),
    'Recession': [1 if 1980 <= year <= 1982 or 2008 <= year <= 2009 else 0 for year in range(1980, 2024)]
})

app = dash.Dash(__name__)

year_list = [i for i in range(1980, 2024)]

app.layout = html.Div(children=[
    html.H1("Automobile Sales Statistics Dashboard", 
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 26}),
    
    dcc.Dropdown(
        id='dropdown-statistics',
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
        ],
        placeholder='Select a report type',
        value="Select Statistics",
        style={'width': '60%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    dcc.Dropdown(
        id="select-year",
        options=[{'label': str(i), 'value': i} for i in year_list],
        placeholder="Select year",
        value="Select year",
        style={'width': '60%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    html.Div([
        html.Div(id="output-container", className="chart-grid", style={
             'display': 'flex', 
             'border': '5px solid black', 
             'width': '80%', 
             'height': '200px', 
             'margin': '20px auto',
             'background-color': '#f0f0f0',
             'justify-content': 'center',
             'align-items': 'center'
         })
    ])
])

@app.callback(
    Output(component_id="select-year", component_property="disabled"),
    Input(component_id="dropdown-statistics", component_property="value")
)
def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics': 
        return False
    else:  
        return True

@app.callback(
    Output(component_id="output-container", component_property="children"),
    [Input(component_id="dropdown-statistics", component_property="value"),
     Input(component_id="select-year", component_property="value")]
)
def update_output_container(statistics_type, selected_year):
    if statistics_type == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]
        return html.Div([
            html.H3("Recession Period Sales Data"),
            dcc.Graph(
                figure={
                    'data': [
                        {'x': recession_data['Year'], 'y': recession_data['Sales'], 'type': 'bar', 'name': 'Sales'}
                    ],
                    'layout': {
                        'title': 'Recession Period Sales'
                    }
                }
            )
        ])
    elif statistics_type == 'Yearly Statistics' and selected_year != "Select year":
        # Filter the data for the selected year
        yearly_data = data[data['Year'] == selected_year]
        return html.Div([
            html.H3(f"Sales Data for {selected_year}"),
            dcc.Graph(
                figure={
                    'data': [
                        {'x': yearly_data['Year'], 'y': yearly_data['Sales'], 'type': 'bar', 'name': 'Sales'}
                    ],
                    'layout': {
                        'title': f'Sales in {selected_year}'
                    }
                }
            )
        ])
    return html.Div("Please select a valid option")

if __name__ == '__main__':
    app.run_server(debug=True)
